<a href="https://colab.research.google.com/github/petermesy/Machine-Learning-Projects/blob/main/Peter_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install pdfplumber
!pip install duckduckgo-search
!pip install beautifulsoup4
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=0be7941a21c4f672a8554dafc54819983b4f25e47e94402ca2f3e5371e5ee59d
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [6]:
import os
import requests
import pdfplumber
from langdetect import detect
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
from urllib.parse import urlparse

import json

In [7]:
DOWNLOAD_FOLDER = "Downloaded_pdfs"
os.makedirs(DOWNLOAD_FOLDER, exist_ok=True)

In [8]:
# Function to search for PDF URLs
def search_pdf_urls(query, max_results=1000):
    pdf_urls = []
    with DDGS() as ddgs:
        for result in ddgs.text(query + " filetype:pdf", max_results=max_results):
            url = result.get("href") or result.get("url")
            if url and url.lower().endswith(".pdf"):
                pdf_urls.append(url)
    return pdf_urls

In [9]:
# Function to download PDF
def download_pdf(url, folder):
    try:
        response = requests.get(url, timeout=None)
        if response.status_code == 200 and 'application/pdf' in response.headers.get('Content-Type', ''):
            filename = os.path.basename(urlparse(url).path)
            filepath = os.path.join(folder, filename)
            with open(filepath, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded: {filename}")
        else:
            print(f"Skipped (not a valid PDF): {url}")
    except Exception as e:
        print(f"Failed to download {url}: {e}")

In [10]:
# Function to extract text from PDF and detect language
def extract_text_and_detect_language(pdf_path):
    try:
        with pdfplumber.open(pdf_path) as pdf:
            text = ""
            for page in pdf.pages:
                text += page.extract_text()
        language = detect(text)
        return text, language
    except Exception as e:
        print(f"Failed to extract text from {pdf_path}: {e}")
        return None, None

In [12]:
# Example usage
if __name__ == "__main__":
    # Change the search query to target Amharic PDFs
    search_query = "የፍትህ አዲስ ደንብ አዲስ ሕግ የፌዴራል ሕጎች filetype:pdf"
    pdf_links = search_pdf_urls(search_query, max_results=1000)

    # Save PDF links to a JSON file
    pdf_links_file = os.path.join(DOWNLOAD_FOLDER, "pdf_links.json")
    with open(pdf_links_file, "w", encoding="utf-8") as json_file:
        json.dump(pdf_links, json_file, ensure_ascii=False, indent=4)
    print(f"PDF links saved to: {pdf_links_file}")





    for link in pdf_links:
        download_pdf(link, DOWNLOAD_FOLDER)

    # Extract text and detect language for each downloaded PDF
    for filename in os.listdir(DOWNLOAD_FOLDER):
        if filename.endswith(".pdf"):
            filepath = os.path.join(DOWNLOAD_FOLDER, filename)
            text, language = extract_text_and_detect_language(filepath)
            if text and language:
                print(f"Extracted Text from {filename}:\n{text[:100]}...")  # Print first 100 characters of extracted text
                output_filename = os.path.splitext(filename)[0] + ".txt"  # Replace .pdf with .txt
                output_filepath = os.path.join(DOWNLOAD_FOLDER, output_filename)
                with open(output_filepath, "w", encoding="utf-8") as text_file:
                    text_file.write(text)
                print(f"Extracted text saved to: {output_filepath}")
                print(f"Detected Language: {language}")


PDF links saved to: Downloaded_pdfs/pdf_links.json
Downloaded: ethiopias-transitional-justice-policy-.pdf
Downloaded: ETH112600.pdf
Skipped (not a valid PDF): https://www.lawethiopia.com/images/addis+ababa/Regulation+Number+125-2022+Addis+Ababa+prosecutors+regulation.pdf
Skipped (not a valid PDF): https://www.lawethiopia.com/images/draft+laws/Draft+Advocate's+Proclamation+-+Amharic+Version.pdf
Downloaded: transitional-justice-draft-stamped.pdf
Downloaded: f2bede9c-9524-42f7-ac88-41d64a735212.pdf
Downloaded: a3bf28be-a8c3-408f-8671-a9adebb3281f.pdf
Downloaded: ETH101059.pdf
Skipped (not a valid PDF): https://addisstandard.com/wp-content/uploads/2018/06/Amensty-Amharic-Version_04_06_18.pdf
Downloaded: e18ba8e18d8ce18bb4e188abe1888d-e18898e18a95e18c8de188b0e189b5-e188a0e188abe189b0e18a9b-e18aa0e18b8be18c85-e18981e18ca5e188ad-1064.pdf
Downloaded: 1d0f5b4a-f4e3-48e2-9ae3-a9c5c714606f.pdf
Skipped (not a valid PDF): https://lawethiopia.com/images/cassation/cassation+decisions+by+number/volume

Failed to extract text from Downloaded_pdfs/Procurement_Directive_Amharic.pdf: No features in text.


Failed to extract text from Downloaded_pdfs/vat_proclamation_latest_draft-_amharic-2023.pdf: No features in text.


Failed to extract text from Downloaded_pdfs/e18ba8e18d8ce18bb4e188abe1888d-e18898e18a95e18c8de188b0e189b5-e188a0e188abe189b0e18a9b-e18aa0e18b8be18c85-e18981e18ca5e188ad-1064.pdf: No features in text.


Failed to extract text from Downloaded_pdfs/Edir-By-law-Amharic.pdf: No features in text.


...

Extracted Text from 258__yafeedeeraale_manegesete_yagezhe_afatzaatzame_mamariyaane_enedaganaa_lamaashaashaale_yawathaa_mamariyaa_amendment_to_the_procurement_directive_final.pdf:
የፌዴራሌ መንግሥት የግዥ አፈፃፀም መመሪያን
እንደገና ሇማሻሻሌ የወጣ መመሪያ ቁጥር 258/2013
የፌዴራሌ መንግሥት የግዥ አፈፃፀም መመሪያን ማሻሻሌ አስፈሊጊ...
Extracted text saved to: Downloaded_pdfs/258__yafeedeeraale_manegesete_yagezhe_afatzaatzame_mamariyaane_enedaganaa_lamaashaashaale_yawathaa_mamariyaa_amendment_to_the_procurement_directive_final.txt
Detected Language: cy


Extracted Text from ETH112600.pdf:
የኢትዮጵያ ፌደራላዊ ዲሞክራሲያዊ ሪፐብሊክ
ፌደራል ነጋሪት ጋዜጣ
FEDERAL NEGARIT GAZETTE
OF THE FEDERAL DEMOCRATIC REPUBLIC ...
Extracted text saved to: Downloaded_pdfs/ETH112600.txt
Detected Language: en
Failed to extract text from Downloaded_pdfs/264_yafeedeeraale_manegeszete_yagezhe_afatzaatzame_mamariyaane_enedaganaa_lamaashaashaale_yawathaa_mamariyaa_quthere_264-2013.pdf: No features in text.


Failed to extract text from Downloaded_pdfs/አደረጃጀት-v28-አዋጅ_AMH-2014.pdf: No features in text.


Extracted Text from 943.pdf:
የኢትዮጵያ ፌደራላዊ ዲሞክራሲያዊ ሪፐብሊክ
ፌደራል ነጋሪት ጋዜጣ
FEDERAL NEGARIT GAZETTE
OF THE FEDERAL DEMOCRATIC REPUBLIC ...
Extracted text saved to: Downloaded_pdfs/943.txt
Detected Language: en


Extracted Text from f2bede9c-9524-42f7-ac88-41d64a735212.pdf:
የኢትዮጵያ ፌደራላዊ ዲሞክራሲያዊ ሪፐብሊክ
ፌደራል ነጋሪት ጋዜጣ
FEDERAL NEGARIT GAZETTE
OF THE FEDERAL DEMOCRATIC REPUBLIC ...
Extracted text saved to: Downloaded_pdfs/f2bede9c-9524-42f7-ac88-41d64a735212.txt
Detected Language: en
